In [49]:
from crewai import Agent, Task, Process, Crew, LLM
from langchain_community.tools import DuckDuckGoSearchRun


import os
from getpass import getpass
from textwrap import dedent

In [50]:
from langchain_community.llms import Ollama
# ollama_llm = Ollama(model="openhermes") # assuming you have Ollama installed and have openhermes model pulled with `ollama pull openhermes`
# ollama_llm = Ollama(model="phi")
# ollama_llm = Ollama(model="llama3")

# ollama_llm = ChatOpenAI(
#     model = "crewai-llama3",
#     base_url = "http://localhost:11434/v1")

# from litellm import completion

# response = completion(
#     model="ollama/crewai-llama3", 
#     messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
#     api_base="http://localhost:11434"
# )
# print(response)
ollama_llm=LLM(model="ollama/crewai-llama3", base_url="http://localhost:11434")

In [51]:
from crewai_tools import WebsiteSearchTool

# Dynamically generate the search query based on the topic
def generate_search_query(topic):
    return f"latest {topic} promotions in Singapore"

# Sample topic
topic = "food"

# Generate the search query dynamically
search_query = generate_search_query(topic)

# Ensure the search query is a string before passing it to WebsiteSearchTool
assert isinstance(search_query, str), f"Expected string, got {type(search_query)}"

# The website you want to search on
website = 'https://www.google.com/'  # @param {type:"string"}

# Initialize WebsiteSearchTool with both search query and website
website_search_tool = WebsiteSearchTool(search_query=search_query, website=website)

Inserting batches in chromadb:   0%|          | 0/1 [00:02<?, ?it/s]


In [52]:
# from crewai_tools import tool
# @tool("Duck_Duck_Go_Search")
# def ddgsearch(question: str) -> str:
#     """Clear description for what this tool is useful for, your agent will need this information to use it."""
#     # Function logic here
#     return DuckDuckGoSearchRun().run(question)

In [53]:
# # What is it that needs to get done?
# research_topic = "Differences between open source large language models and closed source large language models."

In [54]:
# researcher = Agent(
#   role='Researcher',
#   goal='Search the internet for information about {research_topic}',
#   backstory="""
#   You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
#   You provide a lot of information thereby allowing a choice in the content selected for the final blog.
#   """,
#   verbose=True,
#   allow_delegation=False,
#   # tools=[DuckDuckGoSearchRun()],
#   tools=[ddgsearch],
#   llm=ollama_llm
# )

In [55]:
from crewai import Agent


agent_1 = Agent(
    role=dedent((
        """
        A diligent researcher with an eye for detail, responsible for tracking down the latest {topic} promotions from reputable sources. This agent will scour official restaurant websites, {topic} deal platforms, and social media pages to gather accurate, up-to-date promotion information.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Promotion Seeker is a seasoned online sleuth who has spent years following the evolving {topic} scene in Singapore. They are well-versed in finding the best {topic} deals, from 1-for-1 offers to limited-time discounts. Their expertise lies in navigating {topic} deal websites and social media channels, ensuring no valuable promotion goes unnoticed.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        To gather the 50 latest {topic} promotions in Singapore, ensuring that the deals are current and valid, prioritizing official restaurant sources and reputable websites. They aim to be the first to spot new offers and update the list with accurate, reliable information.
        """)), # This is the goal that the agent is trying to achieve
    tools=[website_search_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ollama_llm

)

agent_2 = Agent(
    role=dedent((
        """
        A data-driven expert focused on evaluating and verifying the quality of {topic} promotions. This agent’s job is to cross-check information, ensuring that the promotions are genuine, and that the details provided are consistent across different sources.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Deal Analyzer has an analytical mind, accustomed to filtering out noise and focusing on the most impactful promotions. With years of experience analyzing {topic} deals, they can easily detect when a promotion is too good to be true or when an offer might be misleading. They have extensive knowledge of {topic} trends, typical restaurant promotions, and common marketing strategies in the F&B industry.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        To validate each {topic} promotion's details, ensuring its accuracy, legitimacy, and relevance to the current day. They will ensure that every deal included in the list is genuine, with proper start and end dates, and no expired offers.
        """)), # This is the goal that the agent is trying to achieve
    tools=[website_search_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ollama_llm
)

agent_3 = Agent(
    role=dedent((
        """
        The curator of official sources, this agent focuses on gathering and organizing the best references from trustworthy websites, restaurant menus, and social media channels. They ensure that each {topic} promotion’s source is credible and linked to the original post.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Source Curator has a deep network in the {topic} community, following top Singapore {topic} blogs, restaurant Instagram pages, and {topic} deal aggregators. They know which sources are trustworthy and have an extensive list of bookmarked pages to find reliable information quickly. With a passion for the {topic} industry, they pride themselves on offering only the most authoritative links.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        To gather the most reliable sources for each promotion, ensuring that every offer is directly linked to its original source, whether it be an official restaurant webpage, social media account, or a renowned {topic} deal site.
        """)), # This is the goal that the agent is trying to achieve
    tools=[website_search_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ollama_llm
)

In [56]:
import datetime
from crewai import Task

task_1 = Task(
    description=dedent((
        """
        Your task is to search for the most recent {topic} promotions in Singapore by checking official sources. This includes going through restaurant websites, social media pages (Instagram, Facebook, etc.), and reputable food deal platforms. You need to find details on the latest deals such as 1-for-1 offers, discounts, limited-time set meals, and any other current promotions that are being actively advertised. 
        Focus on ensuring that the promotions you find are valid and up-to-date.
        
        Be sure to check the official pages of well-known restaurants, chains, and {topic} deal platforms. Note down the promotion name, a brief description, and the validity period (e.g., end date or if it’s ongoing). You should prioritize official restaurant sources and recognized food deal websites. Additionally, include the direct URL where the promotion is listed for verification purposes.
        """)),
    expected_output=dedent((
        """
        A list of the 50 most recent and valid {topic} promotions in Singapore, sourced from official and reputable platforms. For each promotion, the following should be included:
        - Name of the restaurant or {topic} brand offering the promotion.
        - A clear description of the promotion (e.g., discounts, 1-for-1 offers, set meals).
        - The promotion's end date or an indication that it is ongoing or until further notice.
        - A link to the official source where the promotion was found.
        
        This list should be up-to-date as of today and should exclude any expired promotions.
        """)),
    agent=agent_1,
    output_file=f'output-files/agent_1-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

task_2 = Task(
    description=dedent((
        """
        Your task is to cross-check the {topic} promotions that were found in the previous task to ensure they are still valid and have not expired. You need to verify the details of each promotion, such as the promotion description, start and end dates, and any specific terms or conditions, by checking multiple reliable sources if necessary. If a promotion has expired or is no longer available, you must remove it from the list.
        
        Additionally, confirm that the promotions are currently being advertised and that there are no significant changes (e.g., new offers added or any discounts being altered). Provide clear evidence of the promotion’s validity, citing sources where necessary.
        """)),
    expected_output=dedent((
        """
        A revised list of the 50 {topic} promotions, ensuring that each promotion is accurate, legitimate, and still valid. The updated list should include:
        - Each promotion's current validity status (e.g., valid or expired).
        - If any promotions have expired, remove them from the list.
        - Update any promotions that may have changed in terms of details (dates, terms, etc.).
        - Include references to the sources where the promotion status was confirmed.
        
        The final output should only include promotions that are valid and relevant as of today.
        """)),
    agent=agent_2,
    output_file=f'output-files/agent_2-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

task_3 = Task(
    description=dedent((
        """
        Your task is to organize the list of promotions gathered in previous tasks into a neat and structured format. Ensure that each promotion is easy to read and includes all the necessary details like the promotion description, validity, and an official source link. You should format the information into a cohesive document that is easy to navigate, using clear headings and bullet points for each promotion.
        
        You will need to ensure that all promotions have proper formatting, including well-structured promotion names, clear descriptions, end dates (if applicable), and corresponding links to official sources. The document should be organized in such a way that it allows readers to quickly find the information they need and verify the authenticity of each promotion.
        """)),
    expected_output=dedent((
        """
        A well-organized list of the 50 most up-to-date {topic} promotions in Singapore, formatted in an easy-to-read style. The final document should include:
        - A clear heading for each promotion.
        - A brief description of the promotion.
        - The promotion’s end date or a note if it is ongoing.
        - A direct link to the source of the promotion (e.g., restaurant page, {topic} deal website).
        
        The final list should be formatted in markdown or similar structure, ready for publication or distribution.
        """)),
    agent=agent_3,
    output_file=f'output-files/agent_3-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

In [57]:
# # @title ⌨️ Define any variables you have and input them
# print("## Welcome to the YOUR_CREW_NAME")
# print('-------------------------------------------')
# var_1 = input("What is the  to pass to your crew?\n"),
# var_2 = input("What is the  to pass to your crew?\n"),
# var_3 = input("What is the  to pass to your crew?\n"),
# print("-------------------------------")

In [58]:
from crewai import Crew, Process

def main():
    # Instantiate your crew with a sequential process
    crew = Crew(
        agents=[agent_1, agent_2, agent_3],
        tasks=[task_1, task_2, task_3],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )


    inputs = {"topic":"Food"}
    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)
    
    return result

if __name__ == "__main__":
  result = main()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 31c157af-facf-4d4b-9190-e236cd2db583                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f77d843d-4a99-499b-8c38-b14dad70d340
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f77d843d-4a99-499b-8c38-b14dad70d340
       Status: Executing Task...
    └── 🤖 Agent: 
        A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
        reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
        media pages to gather accurate, up-to-date promotion information.
        
            Status: In Progress

# Agent: 
## Task: 
Your task is to search for the most recent Food promotions in Singapore by checking official sources. This includes going through restaurant websites, social media pages (Instagram, Facebook, etc.), and reputable food deal platforms. You need to find details on the latest deals such as 1-for-1 offers, discounts, limited-time set meals, and any other current promotions that are being actively advertised. 
Focus on ensuring that the promotions you find are valid and up-to-date.

Be sure to check the official pages of well-known restaurants, chains, and Food deal platforms. Note down the promotion name, a brief description, and the validity period (e.g., end date or if it’s ongoing). You should prioritize official restaurant sources and recognized food deal websites. Additionally, include the direct URL where the promotion is listed for verification purposes.



🚀 Crew: crew
└── 📋 Task: f77d843d-4a99-499b-8c38-b14dad70d340
       Status: Executing Task...
    └── 🤖 Agent: 
        A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
        reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
        media pages to gather accurate, up-to-date promotion information.
        
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: f77d843d-4a99-499b-8c38-b14dad70d340
│      Assigned to: 
│   A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from 
│   reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social media 
│   pages to gather accurate, up-to-date promotion information.
│   
│      Status: ✅ Completed
│   ├── 🤖 Agent: 
│   │   A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
│   │   reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
│   │   media pages to gather accurate, up-to-date promotion information.
│   │   
│   │       Status: In Progress
│   │   └── 🔧 Failed Search in a specific website (42)
│   └── 🤖 Agent: 
│       A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
│       reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
│       media pages to gather accurate, up-to-date promotion information.
│       
│           Status: ✅ Completed
│       └── 🔧 Failed Search in a specific website (44)
├── 📋 Task: 2f9e7dca-fd97-4509-9e33-458af40243a2
│      Assigned to: 
│   A data-driven expert focused on evaluating and verifying the quality of Food promotions. This agent’s job is to
│   cross-check information, ensuring that the promotions are genuine, and that the details provided are consistent
│   across different sources.
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: 
│       A data-driven expert focused on evaluating and verifying the quality of Food promotions. This agent’s job 
│       is to cross-check information, ensuring that the promotions are genuine, and that the details provided are 
│       consistent across different sources.
│       
│           Status: ✅ Completed
│       ├── 🔧 Failed Search in a specific website (46)
│       └── 🔧 Using Manual Research (1)
└── 📋 Task: 045804ea-e5c6-4127-8926-69fa4c0ad673
       Status: Executing Task...
    └── 🤖 Agent: 
        The curator of official sources, this agent focuses on gathering and organizing the best references from 
        trustworthy websites, restaurant menus, and social media channels. They ensure that each Food promotion’s 
        source is credible and linked to the original post.
        
            Status: ✅ Completed
        └── 🔧 Failed Search in a specific website (49)

🚀 Crew: crew
├── 📋 Task: f77d843d-4a99-499b-8c38-b14dad70d340
│      Assigned to: 
│   A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from 
│   reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social media 
│   pages to gather accurate, up-to-date promotion information.
│   
│      Status: ✅ Completed
│   ├── 🤖 Agent: 
│   │   A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
│   │   reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
│   │   media pages to gather accurate, up-to-date promotion information.
│   │   
│   │       Status: In Progress
│   │   └── 🔧 Failed Search in a specific website (42)
│   └── 🤖 Agent: 
│       A diligent researcher with an eye for detail, responsible for tracking down the latest Food promotions from
│       reputable sources. This agent will scour official restaurant websites, Food deal platforms, and social 
│       media pages to gather accurate, up-to-date promotion information.
│       
│           Status: ✅ Completed
│       └── 🔧 Failed Search in a specific website (44)
├── 📋 Task: 2f9e7dca-fd97-4509-9e33-458af40243a2
│      Assigned to: 
│   A data-driven expert focused on evaluating and verifying the quality of Food promotions. This agent’s job is to
│   cross-check information, ensuring that the promotions are genuine, and that the details provided are consistent
│   across different sources.
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: 
│       A data-driven expert focused on evaluating and verifying the quality of Food promotions. This agent’s job 
│       is to cross-check information, ensuring that the promotions are genuine, and that the details provided are 
│       consistent across different sources.
│       
│           Status: ✅ Completed
│       ├── 🔧 Failed Search in a specific website (46)
│       └── 🔧 Using Manual Research (1)
└── 📋 Task: 045804ea-e5c6-4127-8926-69fa4c0ad673
       Assigned to: 
    The curator of official sources, this agent focuses on gathering and organizing the best references from 
    trustworthy websites, restaurant menus, and social media channels. They ensure that each Food promotion’s 
    source is credible and linked to the original post.
    
       Status: ✅ Completed
    └── 🤖 Agent: 
        The curator of official sources, this agent focuses on gathering and organizing the best references from 
        trustworthy websites, restaurant menus, and social media channels. They ensure that each Food promotion’s 
        source is credible and linked to the original post.
        
            Status: ✅ Completed
        └── 🔧 Failed Search in a specific website (49)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 045804ea-e5c6-4127-8926-69fa4c0ad673                                                                     │
│  Agent:                                                                                                         │
│  The curator of official sources, this agent focuses on gathering and organizing the best references from       │
│  trustworthy websites, restaurant menus, and social media channels. They ensure that each Food promotion’s      │
│  source is credible and linked to the original post.                                                            │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 31c157af-facf-4d4b-9190-e236cd2db583                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



########################
## Here is your custom crew run result:
########################

**

To get the latest information about Food promotions in Singapore, you can check out the official websites of popular food establishments like McDonald's, KFC, and Domino's Pizza. You can also visit online platforms such as Burpple, HungryGoWhere, or FoodPanda to find deals and discounts on your favorite foods.

Additionally, you can follow social media accounts of local eateries, restaurants, and cafes to stay updated about their promotions, offers, and menu updates.


In [39]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on a set of information provided by the researcher.',
  backstory="""You are a writer known for your humorous but informative way of explaining. 
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=ollama_llm
)

In [40]:
task1 = Task(
  agent=researcher,
  description=research_topic,
  expected_output="A complete analysis of the {research_topic}, presented as a report.",
)

In [41]:
task2 = Task(
  agent=writer,
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant facts and differences between open-source LLMs and closed-source LLMs.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound simple, and avoid complex words so it doesn't sound AI generated.""",
  expected_output="""A article containing a minimum of 4 paragraphs. contains no more than 2500 words but no less than 1500 words.
  The final output will not contain any extra fluff like "Paragraph 1:" or any action the writer should do.""",
)

In [42]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 for different logging levels
)

2024-06-24 09:01:50,914 - 139688074141696 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [43]:
# Get your crew to work!
result = crew.kickoff()

 [2024-06-24 09:01:51][DEBUG]: == Working Agent: Researcher
 [2024-06-24 09:01:51][INFO]: == Starting Task: Differences between open source large language models and closed source large language models.


> Entering new CrewAgentExecutor chain...
Thought: It's time to start searching for information about the differences between open source large language models and closed source large language models.

Action: Duck_Duck_Go_Search

Action Input: {"q": "what are the key differences between open source and closed source large language models 

I encountered an error while trying to use the tool. This was the error: ddgsearch() got an unexpected keyword argument 'q'.
 Tool Duck_Duck_Go_Search accepts these inputs: Duck_Duck_Go_Search(question: 'string') - Clear description for what this tool is useful for, your agent will need this information to use it.

I will proceed with caution, taking into account the error message from the Duck_Duck_Go_Search tool.

Thought: I need to reformulate m

In [17]:
result

"Open-source and closed-source large language models (LLMs) have distinct differences in terms of accessibility, customization, licensing restrictions, cost, innovation, scalability, and vendor lock-in. Open-source LLMs provide greater freedom, flexibility, and control over code modifications without the constraints of commercial licenses. They are accessible to anyone who wishes to inspect or modify the code, but may require in-house expertise for maintenance and updates.\n\nOpen-source LLMs offer a diverse range of models and approaches due to community contributions, which can result in slower innovation compared to closed-source LLMs developed by large corporations with extensive research and development investments. These corporate solutions often deliver innovative capabilities and industry-leading performance but come with significant licensing fees and potential vendor lock-in concerns.\n\nChoosing between open-source and closed-source LLMs depends on your organization's specif

In [12]:
with open("Output.txt", "w") as text_file:
    print(result, file=text_file)